In [1]:
%load_ext autoreload
%autoreload 2
import os 
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.85'

In [3]:
from nn_ansatz import setup
from nn_ansatz import run_vmc


In [4]:
# using routines

lr, damping, nc = 1e-4, 1e-4, 1e-4
n_pre_it = 500
n_walkers = 512
n_layers = 2
n_sh = 64
n_ph = 16
n_det = 8
n_it = 1000
seed = 1


config = setup(system='LiSolidBCC',
               n_pre_it=0,
               n_walkers=64,
               n_layers=2,
               n_sh=32,
               n_ph=8,
               opt='kfac',
               n_det=2,
               print_every=1,
               save_every=5000,
               lr=lr,
               n_it=1000,
               norm_constraint=nc,
               damping=damping)



version 		 160821
seed 		 369
n_devices 		 1
save_every 		 5000
print_every 		 1
exp_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/160821/junk/kfac_1lr-4_1d-4_1nc-4_m64_s32_p8_l2_det2/run0
events_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/160821/junk/kfac_1lr-4_1d-4_1nc-4_m64_s32_p8_l2_det2/run0/events
models_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/160821/junk/kfac_1lr-4_1d-4_1nc-4_m64_s32_p8_l2_det2/run0/models
opt_state_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/160821/junk/kfac_1lr-4_1d-4_1nc-4_m64_s32_p8_l2_det2/run0/models/opt_state
pre_path 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/pretrained/s32_p8_l2_det2_1lr-4_i0.pk
timing_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolidBCC/160821/junk/kfac_1lr-4_1d-4_1nc-4_m64_s32_p8_l2_det2/run0/events/timing
system 		 LiSolidBCC
r_atoms 		 [[0.25 0.25 0.25]
 [0.75 0.75 0.75]]
z_atoms 		 [3. 3.]
n_el 		 6
n_el_atoms 		 [3 3]
peri

In [5]:
run_vmc(config)

TypeError: run_vmc() got an unexpected keyword argument 'version'

In [7]:
# using core functions 
# Sytem setup

from tqdm import trange

import jax.random as rnd 
import jax.numpy as jnp

from nn_ansatz import create_sampler
from nn_ansatz import create_wf
from nn_ansatz import initialise_params, initialise_d0s, expand_d0s
from nn_ansatz import SystemAnsatz
from nn_ansatz import pretrain_wf
from nn_ansatz import create_energy_fn, create_grad_function
from nn_ansatz import create_natural_gradients_fn, kfac
from nn_ansatz import Logging, load_pk, save_pk



r_atoms = config['r_atoms']
z_atoms = config['z_atoms']
n_el_atoms = config['n_el_atoms']
step_size = config['step_size']
n_el = config['n_el']
load_pretrain = False
pre_path = config['pre_path']
norm_constraint = config['norm_constraint']

key = rnd.PRNGKey(seed)

mol = SystemAnsatz(r_atoms,
                   z_atoms,
                   n_el,
                   n_el_atoms=n_el_atoms,
                   n_layers=n_layers,
                   n_sh=n_sh,
                   n_ph=n_ph,
                   n_det=n_det,
                   step_size=step_size)

wf, kfac_wf, wf_orbitals = create_wf(mol)
params = initialise_params(key, mol)
d0s = expand_d0s(initialise_d0s(mol), n_walkers)

sampler, equilibrate = create_sampler(wf, mol, correlation_length=10)


walkers = mol.initialise_walkers(n_walkers=n_walkers)


System: 
 n_atoms = 1 
 n_up    = 2 
 n_down  = 2 
 n_el    = 4 

Ansatz: 
 n_layers = 2 
 n_det    = 8 
 n_sh     = 64 
 n_ph     = 16 

converged SCF energy = -14.351880476202


In [ ]:
# Pretrain

params, walkers = pretrain_wf(params,
                              wf,
                              wf_orbitals,
                              mol,
                              walkers,
                              n_it=n_pre_it,
                              lr=1e-4,
                              n_eq_it=n_pre_it)

In [10]:
# vmc loop with kfac

grad_fn = create_grad_function(wf, mol)


update, get_params, kfac_update, state = kfac(kfac_wf, wf, mol, params, walkers, d0s,
                                                  lr=lr,
                                                  damping=damping,
                                                  norm_constraint=norm_constraint)


steps = trange(0, n_it, initial=0, total=n_it, desc='training', disable=None)
for step in steps:
    key, subkey = rnd.split(key)

    walkers, acceptance, step_size = sampler(params, walkers, d0s, subkey, step_size)

    grads, e_locs = grad_fn(params, walkers, d0s)

    grads, state = kfac_update(step, grads, state, walkers, d0s)

    state = update(step, grads, state)
    params = get_params(state)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()
    

print('exit')

exit


In [9]:
# periodic boundaries test

for (name, method) in zip(('LiSolid'), ('kfac')):
    config = setup(system=name,
                   pretrain=True,
                   n_pre_it=501,
                   n_walkers=2048,
                   n_layers=2,
                   n_sh=64,
                   n_ph=16,
                   opt=method,
                   n_det=8,
                   print_every=100,
                   save_every=5000,
                   lr=1e-4,
                   n_it=10000,
                   norm_constraint=1e-4,
                   damping=1e-3,
                   exp=True)

    run_vmc(**config)


Pretrain path does not exist, can't load


NameError: name 'exit' is not defined

In [3]:
config = setup(system='LiSolid',
               pretrain=True,
               n_pre_it=501,
               n_walkers=2048,
               n_layers=2,
               n_sh=64,
               n_ph=16,
               opt='kfac',
               n_det=8,
               print_every=100,
               save_every=5000,
               lr=1e-3,
               n_it=1000,
               norm_constraint=1e-4,
               damping=1e-3,
               exp=True,
               name='sampler_fix')

run_vmc(**config)

version 		 200421
seed 		 369
save_every 		 5000
print_every 		 100
exp_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4
events_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events
models_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models
opt_state_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models/opt_state
pre_path 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/pretrained/s64_p16_l2_det8_1lr-4_i501.pk
timing_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events/timing
system 		 LiSolid
r_atoms 		 [[0.5 0.5 0.5]]
z_atoms 		 [3.]
n_el 		 3
n_el_atoms 		 [3]
periodic_boundaries 		 True
cell_basis 		 [[-0

training:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: div got incompatible shapes for broadcasting: (2048, 3, 3), (2048, 3, 14).